In [ ]:
from wand.image import Image
import passporteye
import os

In [ ]:
class Date:
    def __init__(self, passport_date):
        self.day = passport_date[4:6]
        self.month = passport_date[2:4]
        self.year = passport_date[0:2]
        self.date_format = self.day + '/' + self.month + '/' + self.year

In [ ]:
all_passports_data = []

# path of directory of passports should be entered here
file_paths = os.listdir('Passports') 

i = 0  # to iterate through each file
j = 0  # to iterate through each page in pdfs

In [ ]:
while i < len(file_paths):
    file_path = 'Passports\\' + file_paths[i]

    if file_path.endswith('.pdf'):
        pdf = Image(filename=file_path, resolution=300)
        pdfImage = pdf.convert('jpg')  # contains list of jpgs of each page in pdf in sequence attribute
        
        if j < len(pdfImage.sequence):
            page = Image(image=pdfImage.sequence[j])
            page.save(filename=str(j)+'.jpg')  # creating a copy of the pdf page as jpg
            # assigning the location of newly created file to file_path
            file_path = str(j)+'.jpg'
            j += 1
        else:
            i += 1
            j = 0
            continue
    else:
        i += 1

    try:
        raw_data = passporteye.mrz.text.MRZ.to_dict(passporteye.read_mrz(file_path))
    except AttributeError and ValueError:
        print('Invalid Passport')
    else:
        my_data = {
            'Full Name': '',
            'Passport Number': '',
            'Date of Birth': Date(raw_data['date_of_birth']).date_format,
            'Country Code': raw_data['country'],
            'Nationality': raw_data['nationality'],
            'Gender': '',
            'Expiry Date': Date(raw_data['expiration_date']).date_format
        }
        # accurately adding full name to my_data
        my_data['Full Name'] = raw_data['names'].replace('KKK', '').strip() + ' ' + raw_data['surname'].replace('KKK', '').strip()
        
        # accurately adding passport number to my_data
        for char in raw_data['number']:
            if char.isalnum():
                my_data['Passport Number'] += char
        
        # accurately adding gender to my_data
        if raw_data['sex'] is 'M' or raw_data['sex'] is 'F':
            my_data['Gender'] = raw_data['sex']
        else:
            my_data['Gender'] = 'M'
            
        all_passports_data.append(my_data)
        
    if j > 0:  # checking if current file in file_paths is a pdf
        os.remove(file_path)  # deleting copy of pdf page

In [ ]:
print(all_passports_data)